# Code

#### Code - get genotypes

In [ ]:
import pandas as pd
import malariagen_data
import numpy as np
import allel

def vcf_to_siaya_glm_data(vcf_path, df_samples, max_missing_filter=0.20, min_maf_filter=0.02 ,split_multiallelic=True, convert_genotypes=True):
    # Read VCF and create a dictionary
    vcf_df = vcf_to_siaya_df(vcf_path, df_samples, max_missing_filter=max_missing_filter, min_maf_filter=min_maf_filter)
    samples = vcf_df.columns[6:]

    if split_multiallelic:
        vcf_df = split_rows_with_multiple_alleles(vcf_df, samples)

    # Convert genotypes to 0/1/2
    if convert_genotypes:
        vcf_df = convert_genotype_to_alt_allele_count(vcf_df, samples)

    vcf_df = vcf_df.assign(snp_id=lambda x: "snp_" + x.CHROM + ":" + x.POS.astype(str) + "_" + x.ALT)
    vcf_df = vcf_df.set_index('snp_id')
    vcf_df = vcf_df.drop(columns=['CHROM', 'POS', 'FILTER_PASS', 'REF', 'ALT', 'ANN']).T
    vcf_df = pd.concat([df_samples.set_index('sample_id').query('location == "Siaya"'), vcf_df], axis=1)
    
    return vcf_df
    

def vcf_to_siaya_df(vcf_path, df_samples, max_missing_filter=0.20, min_maf_filter=0.02, query=None):
    # Read VCF file
    vcf_dict = allel.read_vcf(vcf_path, fields='*')
    samples = vcf_dict['samples']
    contig = vcf_dict['variants/CHROM'] 
    pos = vcf_dict['variants/POS']
    filter_pass = vcf_dict['variants/FILTER_PASS']
    ref = vcf_dict['variants/REF']
    alt = np.apply_along_axis(lambda x: ','.join(x[x != '']), 1, vcf_dict['variants/ALT'])
    ann = vcf_dict['variants/ANN']
    geno = allel.GenotypeArray(vcf_dict['calldata/GT'])
    
    print(f"Initial number of samples: {len(samples)}")
    print(f"Initial number of SNPs: {geno.shape[0]}")
    
    # filter to Siaya data
    siaya_mask = df_samples.eval("sample_id.str.contains('Siaya')", engine='python')
    sample_mask = np.isin(samples, df_samples[siaya_mask].sample_id)

    geno = geno.compress(sample_mask, axis=1)
    samples = samples[sample_mask]

    if query is not None:
        mask = df_samples.eval(query, engine='python')
        geno = geno.compress(mask, axis=1)
        samples = samples[mask]
    
    print(f"Final number of samples: {len(samples)}")
        
    # Calculate allele counts and apply filters
    ac = geno.count_alleles(max_allele=3)
    seg_mask = ac.is_segregating()
    miss_mask = (geno.is_missing().sum(axis=1) <= int(geno.shape[1]*max_missing_filter))
    maf_mask = ac.to_frequencies()[:, 1:].sum(axis=1) > min_maf_filter
    
    print(f"invariant SNPs removed: {np.sum(~seg_mask)}")
    print(f"SNPs removed due to high missingness (>{max_missing_filter*100}%): {np.sum(~miss_mask)}")
    print(f"SNPs removed due to low MAF (<{min_maf_filter}): {np.sum(~maf_mask)}")
    
    # Apply all filters
    mask = np.logical_and(seg_mask, np.logical_and(miss_mask, maf_mask))
    
    # Compress data based on mask
    geno = geno.compress(mask, axis=0)
    contig = contig[mask]
    pos = pos[mask]
    filter_pass = filter_pass[mask]
    ref = ref[mask]
    alt = alt[mask]
    ann = ann[mask]
    
    print(f"Final number of SNPs after all filters: {geno.shape[0]}")
    
    # Create DataFrame versions of VCF and genotypes
    vcf_df = pd.DataFrame({'CHROM': contig, 'POS': pos, 'FILTER_PASS': filter_pass, 'REF': ref, 'ALT': alt, 'ANN': ann})
    geno_df = pd.DataFrame(geno.to_gt().astype(str), columns=samples)
    
    # Combine VCF and genotype DataFrames
    vcf = pd.concat([vcf_df, geno_df], axis=1)
    
    print(f"Final DataFrame shape: {vcf.shape}")
    
    return vcf

def split_rows_with_multiple_alleles(df, samples):
    # Create an empty list to store the new rows
    new_rows = []
    # Iterate through each row
    for index, row in df.iterrows():
        alt_alleles = row['ALT'].split(',')
        # Check if there are multiple alleles in the ALT field
        if len(alt_alleles) > 1:
            for allele_num, allele in enumerate(alt_alleles):
                # Create a new row for each allele
                new_row = row.copy()
                new_row['ALT'] = allele
                # Update genotype fields
                for col in samples:
                    genotype = row[col]
                    # Split the genotype and process it
                    if genotype != './.':
                        gt_alleles = genotype.split('/')
                        new_gt = ['0' if (int(gt) != allele_num + 1 and gt != '0') else gt for gt in gt_alleles]
                        new_row[col] = '/'.join(new_gt)
                new_rows.append(new_row)
        else:
            new_rows.append(row)
    
    new_df = pd.DataFrame(new_rows).reset_index(drop=True)
    return new_df


def convert_genotype_to_alt_allele_count(df, samples):
    nalt_df = df.copy()
    # Iterate through each row
    for index, row in df.iterrows():
        # Update genotype fields
        for col in samples:
                genotype = row[col]
                if genotype != './.':
                    # Split the genotype and count non-zero alleles
                    alleles = genotype.split('/')
                    alt_allele_count = sum([1 for allele in alleles if allele != '0'])
                    nalt_df.at[index, col] = alt_allele_count
                else:
                    nalt_df.at[index, col] = np.nan

    return nalt_df

#### Code -  run GLM and process results

In [ ]:
def calculate_pseudo_r2(results):
    """
    Calculate different pseudo R² measures for a fitted GLM model
    
    Parameters:
    results : statsmodels GLMResults object
        The fitted model results
        
    Returns:
    dict : Dictionary containing different pseudo R² measures
    """
    ll_null = results.null_deviance / -2
    ll_model = results.deviance / -2
    n = results.nobs
    
    r2_mcfadden = 1 - (ll_model / ll_null)
    r2_coxsnell = 1 - np.exp((2/n) * (ll_null - ll_model))
    r2_nagelkerke = r2_coxsnell / (1 - np.exp((2/n) * ll_null))
    
    return {
        'McFadden R²': r2_mcfadden,
        'Nagelkerke R²': r2_nagelkerke
    }

def glm_all_snps(snp_df):
    import statsmodels.formula.api as smf
    import statsmodels.api as sm
    from statsmodels.stats.multitest import fdrcorrection
    
    dfs = []
    for snp in snp_df.filter(like="snp_").columns:
        glm_data = snp_df[['phenotype', snp]].dropna()
        glm_data[snp] = glm_data[snp].astype(int)
        glm_data.columns = ['phenotype', 'snp']
    
        df_contingency = pd.crosstab(glm_data['phenotype'], glm_data['snp'])
        
        formula = f'phenotype ~ snp'
        logit_model = smf.glm(formula=formula, data=glm_data, family=sm.families.Binomial())
        log_results = logit_model.fit()
        
        # Calculate pseudo R² measures
        r2_values = calculate_pseudo_r2(log_results)
    
        pval_data = results_summary_to_dataframe(log_results)
        # Add R² values to the results
        for r2_name, r2_value in r2_values.items():
            pval_data[r2_name] = r2_value
            
        dfs.append(pval_data.assign(snp=snp))
        
    df_eff = pd.concat(dfs).query("index != 'Intercept'")
    res = fdrcorrection(df_eff['pvals'], alpha=0.05)
    df_eff['fdr'] = res[1]
    df_eff['fdr_sig'] = res[0]
    return process_effect_sizes(df_eff)


def process_effect_sizes(df_eff):
    df_eff = df_eff.assign(CHROM=lambda x: x.snp.str.split(":").str[0].str.replace("snp_", ""), 
                   POS_ALT=lambda x: x.snp.str.split(":").str[1]).assign(POS=lambda x: x.POS_ALT.str.split("_").str[0].astype(int))
    df_eff = df_eff.merge(snp_data[['CHROM', 'POS', 'ANN']])
    cols = ['snp'] + [col for col in df_eff.columns if col != 'snp']
    df_eff = df_eff[cols].drop(columns=['CHROM', 'POS'])
    df_eff = df_eff.set_index('snp')
    return df_eff    


def results_summary_to_dataframe(results):
    '''take the result of an statsmodel results table and transforms it into a dataframe'''
    pvals = results.pvalues
    coeff = results.params
    conf_lower = results.conf_int()[0]
    conf_higher = results.conf_int()[1]

    results_df = pd.DataFrame({"pvals":pvals,
                               "odds_ratio":np.exp(coeff),
                               "conf_lower":np.exp(conf_lower),
                               "conf_higher":np.exp(conf_higher)
                                })
    
    results_df.loc[:, 'sig'] = [True if pval <= 0.05 else False for pval in pvals]
    results_df = results_df[["odds_ratio","pvals","conf_lower","conf_higher", "sig"]]
    return results_df

#### Code - compute correlations and select snps

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def compute_snp_correlations(df, correlation_threshold=0.5):
    corr_matrix = df.corr()
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
   
    high_corr_pairs = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            if abs(corr_matrix.iloc[i, j]) > correlation_threshold:
                high_corr_pairs.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]))
    
    print(f"Total SNPs analyzed: {len(df.columns)}")
    print(f"Number of highly correlated SNP pairs (|r| > {correlation_threshold}): {len(high_corr_pairs)}")
    
    return corr_matrix, high_corr_pairs

def plot_correlation_heatmap(corr_matrix):
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', vmin=-1, vmax=1, center=0)
    plt.title('SNP Correlation Heatmap')
    plt.show()

def select_representative_snps(df, high_corr_pairs):
    to_remove = set()
    pos_corr_removed = 0
    neg_corr_removed = 0
    
    for snp1, snp2, corr in high_corr_pairs:
        if snp1 not in to_remove and snp2 not in to_remove:
            if corr < 0:  # Negative correlation
                if df.loc[snp1, 'odds_ratio'] < 1:
                    to_remove.add(snp1)
                    neg_corr_removed += 1
                elif df.loc[snp2, 'odds_ratio'] < 1:
                    to_remove.add(snp2)
                    neg_corr_removed += 1
                else:
                    if df.loc[snp1, 'fdr'] < df.loc[snp2, 'fdr']:
                        to_remove.add(snp2)
                        neg_corr_removed += 1
                    else:
                        to_remove.add(snp1)
                        neg_corr_removed += 1
            else:  # Positive correlation
                if df.loc[snp1, 'fdr'] < df.loc[snp2, 'fdr']:
                    to_remove.add(snp2)
                    pos_corr_removed += 1
                else:
                    to_remove.add(snp1)
                    pos_corr_removed += 1
    
    selected_snps = np.array(list(set(df.index) - to_remove))
    
    print(f"Total SNPs before selection: {len(df)}")
    print(f"SNPs removed due to positive correlation: {pos_corr_removed}")
    print(f"SNPs removed due to negative correlation: {neg_corr_removed}")
    print(f"Total SNPs removed: {len(to_remove)}")
    print(f"SNPs retained after selection: {len(selected_snps)}")
    
    return selected_snps

def linked(df_genos, df_eff, correlation_threshold=0.5):
    corr_matrix, high_corr_pairs = compute_snp_correlations(df_genos.filter(like='snp_'), correlation_threshold)
    plot_correlation_heatmap(corr_matrix)
    selected_snps = select_representative_snps(df_eff, high_corr_pairs)
    return np.sort(selected_snps)

#### Code - plot PRS scores

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

def plot_snp_frequency(df_snp, selected_snps):
    df = df_snp.loc[:, selected_snps]
    
    contigs = df.columns.str.split("_").str.get(1).str.slice(0,2)
    total_alleles = df.shape[0]
    freq = (df.sum() / (total_alleles * 2)).to_frame().assign(contig=contigs.to_list()).reset_index()
    freq.columns = ['snp_id', 'freq', 'contig']
    fig = px.bar(freq, x='snp_id', y='freq', color='contig', template='simple_white', width=800, height=300)
    fig.show()

# Integrating polygenic risk scores into the LLINEUP ento and epi models
First lets load the metadata for the Siaya amplicon-sequenced samples

In [ ]:
df_amp_samples = pd.read_csv("/home/sanj/Projects/amplicon-seq/ampseq-agvampir002/results/config/metadata.qcpass.tsv", sep="\t")
df_amp_samples = df_amp_samples.assign(phenotype=df_amp_samples.sample_id.str.extract(r'_(Dead|Alive)_'))
df_amp_samples = df_amp_samples.rename(columns={'sampleID':'sample_id'})

### Perform association tests with separate binomial GLMs

In [ ]:
snp_data = pd.read_excel("../resources/ampseq-vigg002-snps-amplicons.xlsx")

In [ ]:
import warnings 
warnings.filterwarnings("ignore", category=RuntimeWarning, module="pandas.core.arraylike")
warnings.filterwarnings("ignore", category=UserWarning, module="allel.io")

df_genos = {}
df_effs = {}
methods = ['targets', 'amplicons']
methods  =  ['amplicons']
for method in methods:
    df_genos[method] = vcf_to_siaya_glm_data(
        vcf_path=f"../resources/ampseq-vigg002.annot.{method}.vcf", 
        df_samples=df_amp_samples, 
        min_maf_filter=0.02, 
        max_missing_filter=0.20
    )
    print("\n")

    # run a glm on each snp on the input data 
    df_effs[method] = glm_all_snps(df_genos[method])
    # df_effs[method].to_csv(f"glm-siaya-effect-size-{method}.csv")

In [ ]:
df_genos['amplicons'].query("taxon  != 'coluzzii'").to_csv("../df_geno_amplicons.csv")

### Compute correlations between snps and prune snps in LD

In [ ]:
selected_snps = {}
for method in methods:
    selected_snps[method] = linked(df_genos[method].filter(like='snp_').dropna().astype(int), df_effs[method])

In [ ]:
for method in methods:
    print(method)
    plot_snp_frequency(df_genos[method], selected_snps=selected_snps[method])